In [1]:
import numpy as np
import pandas as pd
from numpy.random import default_rng, SeedSequence
import scipy.stats as sps
import matplotlib.pyplot as plt
from scipy.stats import norm
import warnings

sq = SeedSequence()
seed = sq.entropy
rng = default_rng(sq)

### Tools

In [2]:
def brownian_1d(n_times: int, n_paths: int, 
                final_time: float=1.0, 
                increments: bool=False, 
                random_state: np.random.Generator=rng) -> np.array:
    """Simulate paths of standard Brownian motion
    Args:
        n_times: Number of timesteps
        n_paths: Number of paths 
        final_time: Final time of simulation
        increments: If `True` the increments of the paths are returned.
        random_state: `np.random.Generator` used for simulation
    Returns:
        `np.array` of shape `(n_times+1, n_paths)` containing the paths if the argument `increments` is `False`
        `np.array` of shape `(n_times, n_paths)` containing the increments if the argument `increments` is `True`
    """
    dB = np.sqrt(final_time / n_times) * random_state.standard_normal((n_times, n_paths))
    if increments:
        return dB
    else:
        brownian = np.zeros((n_times+1, n_paths))
        brownian[1:] = np.cumsum(dB, axis=0)
        return brownian

In [3]:
def black_scholes_1d(n_times: int, n_paths: int, 
                     final_time: float=1.0, 
                     random_state: np.random.Generator=rng, *,
                     init_value: float,
                     r: float, sigma: float) -> np.array:
    """Simulate paths of Black-Scholes process
    Args:
        n_times: Number of timesteps
        n_paths: Number of paths 
        final_time: Final time of simulation
        init_value: `S0`
        r: Interest rate
        sigma: Volatility
        random_state: `np.random.Generator` used for simulation
    Returns:
        `np.array` of shape `(n_times+1, n_paths)` containing the paths 
    """
    Bt = brownian_1d(n_times, n_paths)
    times = np.arange(n_times+1)*(1/n_times)
    t = times[:, np.newaxis]
    St = init_value * np.exp((r - 0.5*sigma**2)*t + sigma*Bt)
    return St

In [4]:
# une fonction BS pour un payoff qui n'est pas path-dependent (offre plus de liberté pour le choix des gaussiennes dans la fonction)
def BS(x,r,sigma,T,N):
    """ args :
            x=spot
            r=interest rate
            sigma=volatility
            T=maturity
            N=simulated standard normal random variable
    """
    return  x*np.exp((r-(sigma**2)/2)*T+sigma*np.sqrt(T)*N)

# une fonction de payoff du call et sa dérivée par rapport à S_T
def payoff_call(S,r,T,K): return np.exp(-r*T)*np.maximum(S-K,0)
def payoff_put(S,r,T,K): return np.exp(-r*T)*np.maximum(K-S,0)

def call_derive(S,r,T,K): return np.exp(-r*T)*np.where(S>K,1,0)

In [5]:
def monte_carlo(sample, proba = 0.95):
    mean = np.mean(sample)
    var = np.var(sample, ddof=1)
    alpha = 1 - proba 
    quantile = norm.ppf(1 - alpha/2)  # fonction quantile 
    ci_size = quantile * np.sqrt(var / sample.size)
    return (mean, var, mean - ci_size, mean + ci_size)

In [6]:
# Les formules fermées de Black-Scholes pour vérifier nos méthodes de MC

def d1(spot, t, r, sigma, strike):
    return (np.log(spot / strike) + t * (r + 0.5*sigma**2)) / (sigma * np.sqrt(t))

def d2(spot, t, r, sigma, strike):
    return d1(spot, t, r, sigma, strike) - sigma * np.sqrt(t)

def price_call_BS(spot, t, r, sigma, strike):
    d1_ = d1(spot, t, r, sigma, strike)
    d2_ = d2(spot, t, r, sigma, strike)
    return spot * norm.cdf(d1_) - strike * np.exp(-r * t) * norm.cdf(d2_)

def delta_BS(spot, t, r, sigma, strike):
    d1_ = d1(spot, t, r, sigma, strike)
    return norm.cdf(d1_)

### Standard MC

In [7]:
# Fixons les paramètres

S0 = 100
T=1
K=100
r, sigma = 0.04, 0.20

In [8]:
Ms = 10**np.arange(3, 8)
results = pd.DataFrame(index=['Mean', 'Var', 'Lower', 'Upper'], columns=Ms)
for M in Ms:
    gaussiennes = rng.standard_normal(M)
    payoffs=payoff_call(BS(x=S0,r=r,sigma=sigma,T=T,N=gaussiennes),r=r,T=T,K=K)
    results[M] = monte_carlo(payoffs)
results

,1000,10000,100000,1000000,10000000
Mean,9.712122,10.042327,9.886496,9.926891,9.926359
Var,189.080850,207.154298,206.833680,208.259954,207.870333
Lower,8.859863,9.760233,9.797359,9.898607,9.917422
Upper,10.564382,10.324422,9.975634,9.955176,9.935295


In [9]:
price_call_BS(S0, T, r, sigma, K)

9.925053717274437

# Basket options

In [21]:
def sim_actifs(d:int, init_value: np.array, vol: np.array, Sigma, expiry, r, n_paths):
    G=rng.standard_normal((d,n_paths))
    L=np.linalg.cholesky(Sigma)
    Gtild=L@G
    ST = init_value[:,np.newaxis] * np.exp((r - 0.5*vol[:,np.newaxis]**2)*expiry +np.sqrt(expiry)*vol[:,np.newaxis]*Gtild)
    return ST

In [22]:
# Exemple : 5 actifs de valeur initiale 100 et de volatilité croissante (en l'actif i), maturité 1 an, taux 2%
n_paths=2
d=5
T=1
r=0.02
S0=np.full(d,100)
K=100
vol=np.linspace(0.2,0.3,5)
rho=0.3
Sigma = np.full((d,d), rho) + (1-rho)*np.eye(d)
sim_actifs(d,S0,vol,Sigma,T,r,n_paths)

array([[ 87.6537058 , 126.92311041],
       [102.71513622, 108.90173716],
       [138.34319517, 100.49557652],
       [ 94.63089577, 118.72116775],
       [143.18372642, 148.32587702]])

In [23]:
# Pricing MC

alpha=np.array([0.1,0.4,0.3,0.05,0.15])

Ms = 10**np.arange(3, 7)
results = pd.DataFrame(index=['Mean', 'Var', 'Lower', 'Upper'], columns=Ms)
for M in Ms:
    prix=sim_actifs(d,S0,vol,Sigma,T,r,M)
    X=np.exp(-r*T)*np.maximum(np.average(prix, weights=alpha, axis=0)-K,0)
    results[M]=monte_carlo(X)
    
results

,1000,10000,100000,1000000
Mean,7.560983,7.686125,7.861601,7.801095
Var,132.827241,134.707611,140.195042,138.595898
Lower,6.846665,7.458644,7.788215,7.778021
Upper,8.275301,7.913605,7.934987,7.824169


### Variance reduction

Compute discounted expectation of $k_T$.

In [24]:
Sig = alpha.T @ Sigma @ Sigma.T @ alpha

In [25]:
var=np.sum([alpha[i]*Sigma[i,:]@Sigma[i,:].T for i in range(d)])
produit=np.product([(S0[j]**alpha[j])*np.exp(-0.5*(var-Sig)*T) for j in range(d)])
produit

41.64974249447475

In [26]:
premium=price_call_BS(spot=produit, t=T, r=r, sigma=np.sqrt(Sig) , strike=K)
premium

6.525796311465067

We create the simulation function to perform MC.

In [27]:
def sim_payoff(d:int, init_value: np.array, vol: np.array, Sigma, expiry, r, n_paths, strike):
    G=rng.standard_normal((d,n_paths))
    L=np.linalg.cholesky(Sigma)
    Gtild=L@G
    ST = init_value[:,np.newaxis] * np.exp((r - 0.5*vol[:,np.newaxis]**2)*expiry +np.sqrt(expiry)*vol[:,np.newaxis]*Gtild)
    return payoff_call(np.dot(alpha.T,ST), r , expiry, strike)-payoff_call(np.exp(np.dot(alpha.T,np.log(ST))), r , expiry, strike)

In [28]:
# Note : les browniens simulés dans k doit être les mêmes que ceux de h.

Ms = 10**np.arange(3, 8)
results = pd.DataFrame(index=['Mean', 'Var', 'Lower', 'Upper'], columns=Ms)
for M in Ms:
    payoffs=sim_payoff(d=d, init_value=S0, vol=vol, Sigma=Sigma, expiry=T, r=r, n_paths=M, strike=K)
    results[M] = monte_carlo(payoffs)
    results[M].iloc[0]+=premium
    results[M].iloc[2:]+=premium
results

,1000,10000,100000,1000000,10000000
Mean,7.426958,7.419988,7.399729,7.406849,7.407057
Var,1.750897,1.879741,1.839140,1.854558,1.852891
Lower,7.344946,7.393116,7.391324,7.404180,7.406213
Upper,7.508970,7.446860,7.408134,7.409518,7.407900
